# 필요 라이브러리

In [2]:
import pyautogui
import time
import pyperclip
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#  좌표 및 함수 설정

In [3]:
# 하나의 row당 클릭 위치
x = [-1372, -1179, -951, -757, -535]
y = [319] * 5

# 뒤로가기 버튼
backward_loc = [-1897, 51]

# 스크롤 내림 버튼
scroll_down = [-8, 1031]

# 첫 시작 시 클릭 위치
first = [-1692, 489]

# url 위치
url_click = [-1529, 48]

# 2회차 전 회차 클릭
Epi2 = [-1335, 627]

# 최신회차가 1회 밖에 없는 경우 대비한 클릭
Epi1 = [-1204, 569]

# 경고 문구 확인 버튼 클릭 (enter1 : 크롬 , enter2: 크롬드라이버)
enter1 = [-874, 244]
enter2 = [583, 268]

# pyautogui이용한 마우스 클릭 복사 붙여넣기
def clipboard():
    pyautogui.hotkey('ctrl', 'c')
    time.sleep(.1)
    return pyperclip.paste()


# 입력받은 각 url에 대해 html파싱
def scrap_html(url):
    request = requests.get(url)
    html = request.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup


# 웹툰 데이터 스크래핑

In [5]:
rows = []
pyautogui.click(first[0], first[1])
for __ in range(10):                       # 10회 반복(10개 row)
    for i in range(5):                     # 하나의 row에 5개 작품

        time.sleep(.5)
        pyautogui.click(x[i], y[i])

        # 해당 웹툰의 url 추출
        pyautogui.tripleClick(url_click[0], url_click[1])
        time.sleep(0.5)
        url = clipboard()

        ## 웹툰 상세 페이지의 제목, 장르, 작가, 구독자 수, 별점, 분량 파싱

        toon_info = scrap_html(url).find_all('div', 'info')
        toon_info = toon_info[0].text.replace('\n', ',').replace('\t', '').split(',')
        genre, title, author = toon_info[1], toon_info[2], toon_info[4]


        cnt = scrap_html(url).find_all('em', 'cnt')  # 구독자 수, 별점
        subscribers_num = cnt[0].text
        grade_num = cnt[1].text

        ## [베스트댓글 좋아요의 수] 2주 전 최신 회차 클릭 및 url 복사
        pyautogui.click(Epi2[0], Epi2[1])

        # 최신회차가 1회 밖에 없는 경우 대비한 클릭
        pyautogui.click(Epi1[0], Epi1[1])
        time.sleep(0.2)

        # 경고팝업 클릭_크롬
        pyautogui.click(enter1[0], enter1[1])

        # 웹툰 회차 url 추출
        pyautogui.tripleClick(-1529, 48)
        url = clipboard()
        time.sleep(0.2)

        # 해당 회차의 분량 추출
        amount = 0
        img_height = scrap_html(url).find_all('img', width=True, height=True)
        for i in img_height:
            if float(i['width']) > 500:
                amount += float(i['height'])
        time.sleep(0.5)
        amount = round(amount, 1)

        row = [title, genre, author, subscribers_num, grade_num, amount]


        # 공감수 동적 데이터라서 셀레니움 이용
        driver = webdriver.Chrome('./chrome/chromedriver.exe')
        driver.get(url)  
        pyautogui.click(enter2[0], enter2[1])  # 경고 팝업창 확인 클릭

        driver.implicitly_wait(1.5)  #로딩시간 후 베스트 댓글의 좋아요 수를 scrap한다
        comments_agree = driver.find_elements_by_css_selector(
            'div > div.u_cbox_tool > div > a.u_cbox_btn_recomm > em')[0].text
        driver.quit()

        row.append(comments_agree)
        print(row)

        ## 뒤로가기 
        pyautogui.doubleClick(backward_loc[0], backward_loc[1], interval=1)

    ## 해당 row모두 수집 후 스크롤 내려서 다음 작품  클릭 
    for __ in range(5):
        pyautogui.click(scroll_down[0], scroll_down[1])
        time.sleep(.5)
print('Finish')

['Lore Olympus', 'Romance', 'Rachel Smythe ', '5M', '9.73', 124411.0, '93979']
['True Beauty', 'Romance', 'Yaongyi ', '6.2M', '9.65', 131236.0, '69573']
['Down To Earth', 'Romance', 'Pookie Senpai ', '1.9M', '9.82', 79278.0, '66212']
['Boyfriends.', 'Slice of life', 'refrainbow ', '691.3K', '9.58', 33750.0, '27426']
['Suitor Armor', 'Fantasy', 'Purpah ', '992.4K', '9.75', 87627.0, '45340']
['Age Matters', 'Romance', 'Enjelicious ', '3.3M', '9.81', 123853.0, '48312']
['I Love Yoo', 'Drama', 'Quimchee ', '4.2M', '9.79', 99317.0, '44910']
['My Deepest Secret', 'Thriller', 'Hanza Art ', '2.2M', '9.71', 103000.0, '56231']
['Men of the Harem', 'Fantasy', 'HereLee / Alphatart ...', '823.6K', '9.73', 96966.0, '43403']
['Castle Swimmer', 'Fantasy', 'Wendy Lian Martin ', '2M', '9.84', 29959.0, '37931']
['The First Night With the Duke', 'Fantasy', 'MSG / Teava ...', '859.1K', '9.67', 74457.0, '47551']
['My Gently Raised Beast', 'Romance', 'Yeoseulki / Early Flower ...', '404.9K', '9.86', 72091.0,

In [7]:
df = pd.DataFrame(rows,
                  columns=[
                      'Title', 'Genre', 'Author', 'Subscribers', 'Grade',
                      'Amount', 'Comments Likes'
                  ])
df_genre = df

# 그림체 column 추가
df_genre['SO'] = 0
df_genre['KE'] = 0
df_genre['ML'] = 0
df_genre['KA'] = 0
df_genre

,Title,Genre,Author,Subscribers,Grade,Amount,Comments Likes,SO,KE,ML,KA
0,Lore Olympus,Sci-fi,Rachel Smythe,5M,9.73,124411.0,93979,0,0,0,0
1,True Beauty,Sci-fi,Yaongyi,6.2M,9.65,131236.0,69573,0,0,0,0
2,Down To Earth,Sci-fi,Pookie Senpai,1.9M,9.82,79278.0,66212,0,0,0,0
3,Boyfriends.,Sci-fi,refrainbow,691.3K,9.58,33750.0,27426,0,0,0,0
4,Suitor Armor,Sci-fi,Purpah,992.4K,9.75,87627.0,45340,0,0,0,0
5,Age Matters,Sci-fi,Enjelicious,3.3M,9.81,123853.0,48312,0,0,0,0
6,I Love Yoo,Sci-fi,Quimchee,4.2M,9.79,99317.0,44910,0,0,0,0
7,My Deepest Secret,Sci-fi,Hanza Art,2.2M,9.71,103000.0,56231,0,0,0,0
8,Men of the Harem,Sci-fi,HereLee / Alphatart ...,823.6K,9.73,96966.0,43403,0,0,0,0
9,Castle Swimmer,Sci-fi,Wendy Lian Martin,2M,9.84,29959.0,37931,0,0,0,0


In [9]:
df1 = pd.read_csv('웹툰_Likes_0520.csv', encoding='utf-8')  #각 웹툰 좋아요 수 저장된 데이터
df2 = df_genre

for i in df2['Title']:

    if len(df1[df1['Title'].str.contains(i)]) > 0:  # df2의 웹툰이 df1에 있을 경우
        Likes = df1[df1['Title'] == i]['Likes'].values[0]  # 해당 웹툰의 좋아요 수
        row_ = df2[df2['Title'] == i].index[0]  # df2의 index를 찾는다.
        df2.loc[row_, 'Total Likes'] = Likes  # df2의 total Likes에 저장한다.
    else:
        pass

print(df2[df2['Total Likes'].isna()])  # 좋아요 수 가져오지 못한 웹툰 있는지 확인
df_Ro = df2
df_Ro['Genre'] = 'Romance'  # 모든 장르 통일
df_Ro.to_csv('./data/[영어]로맨스.csv', encoding='utf-8', index=False)  # csv파일로 저장
df_Ro

Empty DataFrame
Columns: [Title, Genre, Author, Subscribers, Grade, Amount, Comments Likes, SO, KE, ML, KA, Total Likes]
Index: []


,Title,Genre,Author,Subscribers,Grade,Amount,Comments Likes,SO,KE,ML,KA,Total Likes
0,Lore Olympus,Romance,Rachel Smythe,5M,9.73,124411.0,93979,0,0,0,0,37.1M
1,True Beauty,Romance,Yaongyi,6.2M,9.65,131236.0,69573,0,0,0,0,36.4M
2,Down To Earth,Romance,Pookie Senpai,1.9M,9.82,79278.0,66212,0,0,0,0,8.2M
3,Boyfriends.,Romance,refrainbow,691.3K,9.58,33750.0,27426,0,0,0,0,2.6M
4,Suitor Armor,Romance,Purpah,992.4K,9.75,87627.0,45340,0,0,0,0,3.3M
5,Age Matters,Romance,Enjelicious,3.3M,9.81,123853.0,48312,0,0,0,0,22.5M
6,I Love Yoo,Romance,Quimchee,4.2M,9.79,99317.0,44910,0,0,0,0,26.4M
7,My Deepest Secret,Romance,Hanza Art,2.2M,9.71,103000.0,56231,0,0,0,0,11.1M
8,Men of the Harem,Romance,HereLee / Alphatart ...,823.6K,9.73,96966.0,43403,0,0,0,0,2M
9,Castle Swimmer,Romance,Wendy Lian Martin,2M,9.84,29959.0,37931,0,0,0,0,12.6M
